In [2]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 378kB 910kB/s eta 0:00:01
     |████████████████████████████████| 102.7MB 120kB/s eta 0:00:01   |██▍                             | 7.7MB 2.8MB/s eta 0:00:35     |██████████▏                     | 32.7MB 4.5MB/s eta 0:00:16     |██████████████▌                 | 46.5MB 5.2MB/s eta 0:00:11     |██████████████████▏             | 58.3MB 3.9MB/s eta 0:00:12
     |████████████████████████████████| 3.0MB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 163kB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 6.0MB/s eta 0:00:011
     |████████████████████████████████| 51kB 6.5MB/s eta 0:00:011
     |████████████████████████████████| 266kB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 112kB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 8.1MB/s  eta 0:00:01
     |████████████████████████████████| 3.8MB 4.0MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 8.3MB/s eta 0:00:01
     |█████

  Stored in directory: /Users/dasha_awesome/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
  Stored in directory: /Users/dasha_awesome/Library/Caches/pip/wheels/2f/ae/a0/126678adc5f1f8905309a4712ea29537999787f610edbcb0a4
  Stored in directory: /Users/dasha_awesome/Library/Caches/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
  Stored in directory: /Users/dasha_awesome/Library/Caches/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: /Users/dasha_awesome/Library/Caches/pip/wheels/a7/15/a0/0a0561549ad11cdc1bc8fa1191a353efd30facf6bfb507aefc
  Stored in directory: /Users/dasha_awesome/Library/Caches/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
  Stored in directory: /Users/dasha_awesome/Library/Caches/pip/wheels/d7/de/2e/efa132238792efb6459a96e85916ef8597fcb3d2ae51590dfd
  Stored in directory: /Users/dasha_awesome/Library/Caches/pip/wheels/5c/2e/7e/a1d4d4fcebe

In [12]:
import pickle
import time as tm
import numpy as np
# import argparse
import re

from envs import TradingEnv
from agent import DQNAgent
from utils import get_data, get_scaler, maybe_make_dir

In [13]:
class args:
    mode = 'train'
    initial_invest = 20000
    episode = 200
    batch_size = 32
    weights = '{}-dqn.h5'

maybe_make_dir('weights')
maybe_make_dir('portfolio_val')

In [14]:
timestamp = tm.strftime('%Y%m%d%H%M')

data = np.around(get_data())
train_data = data[:, :3526]
test_data = data[:, 3526:]

env = TradingEnv(train_data, args.initial_invest)
state_size = env.observation_space.shape
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
scaler = get_scaler(env)

portfolio_value = []

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                256       
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 27)                891       
Total params: 2,203
Trainable params: 2,203
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
if args.mode == 'test':
    # remake the env with test data
    env = TradingEnv(test_data, args.initial_invest)
    # load trained weights
    agent.load(args.weights)
    # when test, the timestamp is same as time when weights was trained
    timestamp = re.findall(r'\d{12}', args.weights)[0]

In [16]:
for e in range(args.episode):
    state = env.reset()
    state = scaler.transform([state])
    
    for time in range(env.n_step):
        action = agent.act(state)
        next_state, reward, done, info = env.step(action)
        next_state = scaler.transform([next_state])
        if args.mode == 'train':
            agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, episode end value: {}".format(e + 1, args.episode, info['cur_val']))
            portfolio_value.append(info['cur_val']) # append episode end portfolio value
            break
        if args.mode == 'train' and len(agent.memory) > args.batch_size:
            agent.replay(args.batch_size)
    if args.mode == 'train' and (e + 1) % 10 == 0:  # checkpoint weights
        agent.save('weights/{}-dqn.h5'.format(timestamp))

# save portfolio value history to disk
with open('portfolio_val/{}-{}.p'.format(timestamp, args.mode), 'wb') as fp:
    pickle.dump(portfolio_value, fp)

episode: 1/200, episode end value: 37533.0
episode: 2/200, episode end value: 19057.0
episode: 3/200, episode end value: 19278.0
episode: 4/200, episode end value: 12777.0
episode: 5/200, episode end value: 3476.0
episode: 6/200, episode end value: 52455.0
episode: 7/200, episode end value: 8832.0
episode: 8/200, episode end value: 9177.0
episode: 9/200, episode end value: 4554.0
episode: 10/200, episode end value: 14485.0
episode: 11/200, episode end value: 16791.0
episode: 12/200, episode end value: 13694.0
episode: 13/200, episode end value: 6864.0
episode: 14/200, episode end value: 6372.0
episode: 15/200, episode end value: 24884.0
episode: 16/200, episode end value: 10187.0
episode: 17/200, episode end value: 8553.0
episode: 18/200, episode end value: 13670.0
episode: 19/200, episode end value: 22494.0
episode: 20/200, episode end value: 15614.0
episode: 21/200, episode end value: 32037.0
episode: 22/200, episode end value: 45438.0
episode: 23/200, episode end value: 3102.0
episo

episode: 186/200, episode end value: 31070.0
episode: 187/200, episode end value: 23773.0
episode: 188/200, episode end value: 25605.0
episode: 189/200, episode end value: 14935.0
episode: 190/200, episode end value: 63729.0
episode: 191/200, episode end value: 30720.0
episode: 192/200, episode end value: 16664.0
episode: 193/200, episode end value: 23043.0
episode: 194/200, episode end value: 11388.0
episode: 195/200, episode end value: 16071.0
episode: 196/200, episode end value: 28051.0
episode: 197/200, episode end value: 20252.0
episode: 198/200, episode end value: 8427.0
episode: 199/200, episode end value: 13324.0
episode: 200/200, episode end value: 59291.0
